In [1]:

import numpy as np

import torch
import pytorch3d
from pytorch3d.io import load_obj, save_obj, load_objs_as_meshes
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.ops import sample_points_from_meshes #,knn_points, estimate_pointcloud_normals, knn_gather

import trimesh

import pyvista as pv
pv.start_xvfb()
pv.set_jupyter_backend('html')


In [2]:
from ops.utils import *
from ops.mesh_geometry import *

In [3]:
# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

For Mesh

In [4]:
trg_mesh = load_objs_as_meshes(["data_example/Kitty.obj"], device=device)
points_np = trg_mesh.verts_packed().cpu().numpy()
faces_np = trg_mesh.faces_packed().cpu().numpy()
mesh_np = trimesh.Trimesh(vertices=points_np, faces=faces_np)

/home/yihao/anaconda3/envs/meshgpt_base/lib/python3.9/site-packages/pytorch3d/io/obj_io.py:548: UserWarning: Mtl file does not exist: data_example/Kitty.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


In [5]:
curvature, euler = gaussian_curvature(trg_mesh, return_topology=True)

print('Euler:', euler)

Euler: tensor(-0.0011, device='cuda:0')


In [6]:
dual_area = dual_area_vertex(trg_mesh)
curvature_vertex = curvature.view(1,-1)*dual_area.view(1,-1)
# discrete Gauss-Bonnet theorem
print("Gauss-Bonnet theorem: integral of gaussian_curvature - 2*pi*X = ",(curvature_vertex).sum().cpu().numpy() - 2*np.pi*mesh_np.euler_number)

Gauss-Bonnet theorem: integral of gaussian_curvature - 2*pi*X =  -0.006661891937255859


In [7]:
pl = pv.Plotter(notebook=True)

rescaled_curv = torch.tanh(curvature_vertex*100)
pl.add_mesh(mesh_np, scalars=rescaled_curv.cpu().numpy(), show_edges=True, edge_color='white', cmap='viridis', clim=[-1,1], line_width=0.001)

pl.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

For PCL

In [10]:
from ops.pcl_geometry import *

In [11]:
trg_mesh = normalize_mesh(trg_mesh)

trg_mesh_trimesh = trimesh.Trimesh(trg_mesh.verts_packed().cpu().numpy(), trg_mesh.faces_packed().cpu().numpy())


In [18]:
num_samples = 100000
pointscloud, normals_gt = sample_points_from_meshes(trg_mesh, num_samples, return_normals=True)

In [19]:
pcl_with_frames = Differentiable_Global_Geometry_PointCloud(pointscloud, k = 10, normals= normals_gt)

gaussian_curvature = pcl_with_frames.gaussian_curvature().view(-1)

# trace of the Weingarten map
mean_curvature = pcl_with_frames.weingarten_fields().diagonal(offset=0, dim1=-1, dim2=-2).mean(-1).view(-1)

area = pcl_with_frames.local_voronoi_area().view(-1)



In [20]:
for i in range(5,10):
    pcl_with_frames = Differentiable_Global_Geometry_PointCloud(pointscloud, k = 5*i, normals= normals_gt)
    print('Estimated Genus',  1-torch.round(pcl_with_frames.differentiable_euler_number()[0]/2).item())

    print('Estimated euler',  pcl_with_frames.differentiable_euler_number()[0].item())



Estimated Genus 1.0
Estimated euler -0.27381613850593567
Estimated Genus 1.0
Estimated euler -0.17997273802757263
Estimated Genus 1.0
Estimated euler -0.10879017412662506
Estimated Genus 1.0
Estimated euler -0.08370624482631683
Estimated Genus 1.0
Estimated euler -0.029316892847418785


In [15]:
### Plot the point cloud with the estimated gaussian curvature
pl = pv.Plotter(notebook=True)
rescaled_curv = torch.tanh(gaussian_curvature*area*2000) # rescale the curvature for visualization
pl.add_points(pcl_with_frames.pointscloud.detach().cpu().numpy()[0], scalars=rescaled_curv.detach().cpu().numpy(), point_size=5,cmap="viridis")
pl.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [16]:
### Plot the point cloud with the estimated mean curvature
pl = pv.Plotter(notebook=True)
rescaled_curv = torch.tanh(mean_curvature*area*2000) # rescale the curvature for visualization
pl.add_points(pcl_with_frames.pointscloud.detach().cpu().numpy()[0], scalars=rescaled_curv.detach().cpu().numpy(), point_size=5,cmap="viridis")
pl.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…